This is an initial step in our bootstrapping loop, whereby we use the canonical transcriptions as "prediction" to preannotate our data. We feed these into a tts and judge its output based on acceptability of approximate l2 learner speech.

In [1]:
# imports
import os, json
import pandas as pd

In [ ]:
rel_path = "data/teanglann/irish_data.csv"
data_path = os.path.join(os.getcwd(), "..", "..", rel_path)
print(data_path)

/home/pccady/Studies/ThesisProject/scripts/tts/../../data/teanglann/irish_data.csv


In [3]:
df = pd.read_csv(data_path)
df.head()

,file,phonemes
0,litiam.wav,lʲ i tʲ ia mˠ
1,codail.wav,ˈ k o d̪ˠ ə lʲ
2,caeisiam.wav,k eː ʃ ia mˠ
3,aspalóid.wav,ˈ a sˠ pˠ ə l̻ˠ o dʲ
4,ceifileapód.wav,c e fʲ i lʲ ə pˠ o d̪ˠ


In [5]:
# config
audio_base_url = "gs://canonical_pron/wav_files/"    # gcs bucket path with prefix
output_path = "annotation_tasks.json"

# build task jsons according to https://labelstud.io/guide/task_format
tasks = []
for _, row in df.iterrows():
    filename = row["file"]
    phonemes = row["phonemes"]
    word_id = os.path.splitext(filename)[0] # strip .wav extension

    task = {                        # task format: https://labelstud.io/guide/task_format
        "data": {
            "audio_canon": f"{audio_base_url}{filename}",
            "word": word_id
        },
        "predictions": [
            {
                "model_version": "initial",
                "result": [
                    {
                        "from_name": "transcription",    # name of tag to label the region
                        "to_name": "audio_tts",          # name of tag that provided region to be labeled
                        "type": "textarea",         # type of tag used to annotate task
                        "value": {                  # tag-specific value that includes details of the result of labeling the task
                            "text": [phonemes]
                        }
                    }
                ]
            }
        ]
    }
    tasks.append(task)

# write to json file
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(tasks, f, ensure_ascii=False, indent=2)

print(f"wrote {len(tasks)} tasks to {output_path}")


wrote 19226 tasks to annotation_tasks.json
